# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
cities = pd.read_csv('cities_df.csv')
cities = cities.drop(['Unnamed: 0'],axis=1)
cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mataura,100,NZ,1580548551,63,-46.1927,168.8643,71.01,11.99
1,Bredasdorp,90,ZA,1580547935,73,-34.5322,20.0403,75.99,5.99
2,Sorland,75,NO,1580548435,80,67.6670,12.6934,39.20,24.16
3,Tura,0,IN,1580548155,37,25.5198,90.2201,75.24,6.80
4,Hobart,75,AU,1580548144,93,-42.8794,147.3294,64.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
gmaps.configure(g_key)

fig = gmaps.figure(map_type='HYBRID')

locations = cities[['Lat','Lng','Humidity']]
heatmap_layer = gmaps.heatmap_layer(locations[['Lat','Lng']], weights = locations['Humidity'],max_intensity=30, point_radius=3.0)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
cities.columns


Index(['City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Lat', 'Lng',
       'Max Temp', 'Wind Speed'],
      dtype='object')

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
fit_citiesdf = cities.loc[(cities['Cloudiness'] == 0) & (cities['Humidity'] < 50) &
           (cities['Max Temp']<75) & (cities['Wind Speed'] < 10), :]

fit_citiesdf = fit_citiesdf.reset_index().drop(['index'],axis=1)
fit_citiesdf

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Guerrero Negro,0,MX,1580548489,35,27.9769,-114.0611,61.86,9.26
1,Umm Lajj,0,SA,1580548143,15,25.0213,37.2685,73.35,2.82
2,Sardarshahr,0,IN,1580548156,27,28.4333,74.4833,68.00,3.94
3,Sur,0,OM,1580548145,41,22.5667,59.5289,73.74,5.91
4,Chenghai,0,CN,1580548153,39,23.4632,116.7680,64.40,8.95
5,Khalabat,0,PK,1580548154,39,34.0600,72.8894,62.60,4.70
6,Chhatapur,0,IN,1580548159,30,26.2167,87.0167,73.31,6.64
7,El Reno,0,US,1580548505,44,35.5323,-97.9550,36.00,6.93
8,Ures,0,MX,1580548489,33,29.4333,-110.4000,55.00,3.00
9,Petropavlovsk-Kamchatskiy,0,RU,1580548168,46,53.0452,158.6483,8.60,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df = fit_citiesdf
hotel_df['Hotel Name'] = ''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Guerrero Negro,0,MX,1580548489,35,27.9769,-114.0611,61.86,9.26,
1,Umm Lajj,0,SA,1580548143,15,25.0213,37.2685,73.35,2.82,
2,Sardarshahr,0,IN,1580548156,27,28.4333,74.4833,68.00,3.94,
3,Sur,0,OM,1580548145,41,22.5667,59.5289,73.74,5.91,
4,Chenghai,0,CN,1580548153,39,23.4632,116.7680,64.40,8.95,
5,Khalabat,0,PK,1580548154,39,34.0600,72.8894,62.60,4.70,
6,Chhatapur,0,IN,1580548159,30,26.2167,87.0167,73.31,6.64,
7,El Reno,0,US,1580548505,44,35.5323,-97.9550,36.00,6.93,
8,Ures,0,MX,1580548489,33,29.4333,-110.4000,55.00,3.00,
9,Petropavlovsk-Kamchatskiy,0,RU,1580548168,46,53.0452,158.6483,8.60,4.47,


In [ ]:
# Get list of coordinates 

citylistcoord = # to be updated 

# Call Google Place API

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map